In [1]:
import torch
from   sklearn.datasets import make_classification

In [6]:
X, y = make_classification(n_samples=1_000, random_state=32)

modelo_1

In [72]:
from torch.nn import functional as f 
class modelo_1(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.input_ = torch.nn.Linear(20, 10)
        self.hidden1_ = torch.nn.Linear(10, 5)
        self.hidden2_ = torch.nn.Linear(5, 1)
    
    def forward(self, x):
        x = self.input_(x)
        x = torch.nn.Sigmoid()(x)
        x = self.hidden1_(x)
        x = torch.nn.Sigmoid()(x)
        x = self.hidden2_(x)
        return torch.nn.Sigmoid()(x)

In [73]:
modelo_1 = modelo_1()

In [82]:
modelo_1.input_.weight[0, :]

tensor([-0.2753, -0.0745, -0.1861,  0.0675, -0.1676, -0.0552,  0.1291,  0.2044,
        -0.2425,  0.1455, -0.2151,  0.0831,  0.4080, -0.3007, -0.0881,  0.4269,
        -0.2494,  0.1093, -0.3272, -0.2793], grad_fn=<SliceBackward0>)

In [83]:
def initializer_(x:torch.nn.Linear):
    if isinstance(x, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(x.weight)

modelo_1.apply(initializer_)

modelo_1(
  (input_): Linear(in_features=20, out_features=10, bias=True)
  (hidden1_): Linear(in_features=10, out_features=5, bias=True)
  (hidden2_): Linear(in_features=5, out_features=1, bias=True)
)

In [86]:
modelo_1.input_.weight[0, :]

tensor([ 0.0136, -0.0076,  0.0647,  0.2691,  0.3477,  0.1434, -0.0677, -0.3407,
         0.1523, -0.2532, -0.3901,  0.0640,  0.2347, -0.0496, -0.2522, -0.3954,
        -0.1825, -0.4135,  0.2152, -0.0947], grad_fn=<SliceBackward0>)

modelo 2

In [58]:
modelo_2 = torch.nn.Sequential(
    torch.nn.Linear(20, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, 5),
    torch.nn.ReLU(),  
    torch.nn.Linear(5, 1),
    torch.nn.ReLU()
)
modelo_2

Sequential(
  (0): Linear(in_features=20, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=5, bias=True)
  (3): ReLU()
  (4): Linear(in_features=5, out_features=1, bias=True)
  (5): ReLU()
)

In [59]:
modelo_2[0].weight[0, :]

tensor([ 0.0018,  0.0072,  0.1500,  0.0936,  0.1553, -0.2091,  0.1374,  0.0264,
        -0.1027,  0.1062, -0.0126,  0.1453,  0.1959, -0.0467, -0.0911,  0.1989,
        -0.1247,  0.1934, -0.1918,  0.0253], grad_fn=<SliceBackward0>)

In [60]:
def initializer_(x:torch.nn.Linear):
    if isinstance(x, torch.nn.Linear):
        return torch.nn.init.xavier_uniform_(x.weight)

modelo_2.apply(initializer_)

Sequential(
  (0): Linear(in_features=20, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=5, bias=True)
  (3): ReLU()
  (4): Linear(in_features=5, out_features=1, bias=True)
  (5): ReLU()
)

In [61]:
modelo_2[0].weight[0, :]

tensor([ 0.4316,  0.2650,  0.3772, -0.3212, -0.3830, -0.0953,  0.4137,  0.1022,
        -0.3870,  0.2182, -0.1146,  0.0151,  0.1313, -0.0453, -0.1934, -0.0407,
        -0.4410,  0.0272,  0.2585,  0.3636], grad_fn=<SliceBackward0>)